In [1]:
import http.client
import json
from settings import API_KEY
import time
import  csv
from pandas.io.json import json_normalize
import pandas as pd

In [2]:
def get_url_credits(movie_id, api_key):
    return("/3/movie/" + str(movie_id) + "/credits" + "?api_key=" + str(api_key))

def get_json(conn, url):
	payload = "{}"
	conn.request("GET", url, payload)
	res = conn.getresponse()
	data = res.read()
	return(json.loads(data.decode("    utf-8")))

## Essa parte está OK

In [25]:
def get_movie_credits(ids):
    
	conn = http.client.HTTPSConnection("api.themoviedb.org")    
	movie_credits_df = pd.DataFrame()
    
	for movie_id in ids:
		url = get_url_credits(movie_id, API_KEY)
        
		try:
			credits_data = get_json(conn, url)
		except Exception as e:
			if (credits_data['status_code'] == 25):
				time.sleep(10)                
				credits_data = get_json(conn, url)        
        
		df = json_normalize(credits_data)
		movie_credits_df = movie_credits_df.append(df)

	return movie_credits_df

## Aqui dá problema a partir de um número de ids (o teste com 5 funcionou bem)

In [168]:
def get_cast(ids, movie_credits_df):

    movie_cast_df = pd.DataFrame()

    for idx,movie_id in enumerate(ids,40):

        movie_item = pd.DataFrame()
        movie_item = movie_credits_df[movie_credits_df['id']==int(movie_id)].reset_index()
        print("%s of %s" % (idx,len(ids)))

        print(movie_item['cast'])
        
        for cast_id in movie_item['cast'][0]: # ESSA LINHA ESTÁ DANDO PROBLEMA DE INDEX            
            cast_id.update({'movie_id': int(movie_id)})
            cast_id.pop('profile_path', None)
            movie_cast_df = movie_cast_df.append(cast_id, ignore_index=True)
    
    return movie_cast_df

In [162]:
ids[80]

'10384'

In [151]:
movie_credits_df_2.iloc[40]

cast    [{'profile_path': '/5himGJzSuLoHwqacTz7sXWqgeM...
crew    [{'job': 'Director', 'gender': 2, 'credit_id':...
id                                                   2085
Name: 55, dtype: object

In [173]:
movie_credits_df.iloc[40]

cast    NaN
crew    NaN
id      NaN
Name: 40, dtype: object

In [152]:
# TESTE

movie_cast_df = pd.DataFrame()

for movie_id in ['77', '98', '1359', '8358', '7443','2085']:
#for movie_id in ['77', '98']:    

    movie_item = movie_credits_df[movie_credits_df['id']==int(movie_id)].reset_index()

    for cast_id in movie_item['cast'][0]:
        cast_id.update({'movie_id': int(movie_id)})
        cast_id.pop('profile_path', None)
        movie_cast_df = movie_cast_df.append(cast_id, ignore_index=True)

In [155]:
movie_credits_df[movie_credits_df['id'].isnull()].shape

(26737, 3)

In [156]:
movie_credits_df_2 = movie_credits_df[movie_credits_df['id'].isnull()==0]

In [158]:
movie_credits_df_2 = movie_credits_df_2.drop_duplicates(subset='id', keep="first")

In [159]:
movie_credits_df_2.shape

(115908, 3)

,cast_id,character,credit_id,gender,id,movie_id,name,order
0,4.0,Leonard,52fe4214c3a36847f80024db,2.0,529.0,77.0,Guy Pearce,0.0
1,5.0,Natalie,52fe4214c3a36847f80024df,1.0,530.0,77.0,Carrie-Anne Moss,1.0
2,6.0,Teddy Gammell,52fe4214c3a36847f80024e3,2.0,532.0,77.0,Joe Pantoliano,2.0
3,7.0,Burt,52fe4214c3a36847f80024e7,2.0,534.0,77.0,Mark Boone Junior,3.0
4,10.0,Sammy,52fe4214c3a36847f80024f3,2.0,537.0,77.0,Stephen Tobolowsky,4.0
5,11.0,Mrs. Jankis,52fe4214c3a36847f80024f7,1.0,538.0,77.0,Harriet Sansom Harris,5.0
6,13.0,Dodd,52fe4214c3a36847f80024ff,2.0,540.0,77.0,Callum Keith Rennie,6.0
7,16.0,Jimmy,52fe4214c3a36847f800250b,2.0,544.0,77.0,Larry Holden,7.0
8,9.0,Leonard's Wife,52fe4214c3a36847f80024ef,1.0,536.0,77.0,Jorja Fox,8.0
9,8.0,Waiter,52fe4214c3a36847f80024eb,2.0,535.0,77.0,Russ Fega,9.0


In [7]:
def get_crew(ids, movie_credits_df):

    movie_crew_df = pd.DataFrame()

    for movie_id in ids:

        movie_item = movie_credits_df[movie_credits_df['id']==int(movie_id)]

        for cast_id in movie_item['crew'][0]:
            cast_id.update({'movie_id': movie_id})
            cast_id.pop('profile_path', None)
            movie_crew_df = movie_crew_df.append(cast_id, ignore_index=True)
            
    return movie_crew_df

## Depois precisa encapsular isso em uma Main()

In [19]:
with open('out.csv', 'r') as f: # o 'out.csv' é uma cópia com número reduzido de ids
    reader = csv.reader(f, skipinitialspace=True, delimiter=',')
    ids_list = list(reader)
    
ids = [item for sublist in ids_list for item in sublist] 

In [9]:
movie_credits_df = get_movie_credits(ids)

In [10]:
movie_cast_df = get_cast(ids, movie_credits_df)

In [11]:
movie_crew_df = get_crew(ids, movie_credits_df)

In [12]:
movie_credits_df.head()

,cast,crew,id
0,"[{'gender': 2, 'credit_id': '52fe4214c3a36847f...","[{'job': 'Co-Executive Producer', 'gender': 2,...",77
0,"[{'gender': 2, 'credit_id': '52fe4217c3a36847f...","[{'job': 'Unit Publicist', 'gender': 0, 'credi...",98
0,"[{'gender': 2, 'credit_id': '52fe42eec3a36847f...","[{'job': 'Casting', 'gender': 2, 'credit_id': ...",1359
0,"[{'gender': 2, 'credit_id': '52fe44a2c3a36847f...","[{'job': 'Director', 'gender': 2, 'credit_id':...",8358
0,"[{'gender': 2, 'credit_id': '52fe447cc3a36847f...","[{'job': 'Director', 'gender': 2, 'credit_id':...",7443


In [13]:
movie_cast_df.head()

,cast_id,character,credit_id,gender,id,movie_id,name,order
0,4.0,Leonard,52fe4214c3a36847f80024db,2.0,529.0,77,Guy Pearce,0.0
1,5.0,Natalie,52fe4214c3a36847f80024df,1.0,530.0,77,Carrie-Anne Moss,1.0
2,6.0,Teddy Gammell,52fe4214c3a36847f80024e3,2.0,532.0,77,Joe Pantoliano,2.0
3,7.0,Burt,52fe4214c3a36847f80024e7,2.0,534.0,77,Mark Boone Junior,3.0
4,10.0,Sammy,52fe4214c3a36847f80024f3,2.0,537.0,77,Stephen Tobolowsky,4.0


In [14]:
movie_crew_df.head()

,credit_id,department,gender,id,job,movie_id,name
0,5669f76292514173ff006a6d,Production,2.0,44633.0,Co-Executive Producer,77,Christopher Ball
1,5669f77d92514173ff006a70,Production,2.0,44634.0,Co-Executive Producer,77,William Tyrer
2,5669f7e0c3a3682e98001a71,Production,1.0,556.0,Associate Producer,77,Emma Thomas
3,5669f808c3a3682647001c96,Art,1.0,75391.0,Set Decoration,77,Danielle Berman
4,5669f8189251415ec50023ee,Costume & Make-Up,1.0,47294.0,Costume Design,77,Cindy Evans


In [22]:
with open('out_original.csv', 'r') as f: # o 'out_original.csv' é o arquivo do Daniel com 147 mil ids
    reader = csv.reader(f, skipinitialspace=True, delimiter=',')
    ids_list = list(reader)
    
ids = [item for sublist in ids_list for item in sublist] 

In [26]:
start_time = time.time()
movie_credits_df = get_movie_credits(ids)
print("--- Movie Credits: %s seconds ---" % (time.time() - start_time))
print("")

--- Movie Credits: 30646.00852203369 seconds ---



In [ ]:
movie_credits_df = movie_credits_df.reset_index().drop(['index','status_code','status_message'], axis=1)

In [104]:
movie_credits_df.to_csv('movie_credits_df.csv')

## Aqui, a chamada da get_cast() dá merda

In [180]:
ids_2 = ids[39:]

In [181]:
start_time = time.time()
movie_cast_df = get_cast(ids_2, movie_credits_df_2)
movie_cast_df.to_csv('movie_cast_df.csv')
print("--- Movie Cast: %s seconds ---" % (time.time() - start_time))
print("")

40 of 147468
0    [{'gender': 2, 'credit_id': '52fe4525c3a36847f...
Name: cast, dtype: object
41 of 147468
Series([], Name: cast, dtype: object)


IndexError: index out of bounds

In [ ]:
start_time = time.time()
movie_crew_df = get_crew(ids, movie_credits_df)
movie_crew_df.to_csv('movie_crew_df.csv')
print("--- Movie Crew: %s seconds ---" % (time.time() - start_time))
print("")

In [211]:
movie_credits_df_3 = movie_credits_df.dropna(axis=0, how='any', subset=['id'])

movie_credits_df_3['id'] = movie_credits_df_3['id'].astype(int)
movie_cast_df = pd.DataFrame()

for i in range(movie_credits_df.shape[0]):
    movie_item = pd.DataFrame()
    movie_row = movie_credits_df.iloc[i]

    #movie_item = movie_credits_df[movie_credits_df['id']==int(movie_id)].reset_index()
    print(movie_row['cast'][0]) #é uma str, tem que passar para dict

    print(movie_row['cast'][0].dtype)
    
    for cast_id in movie_row['cast'][0]: # ESSA LINHA ESTÁ DANDO PROBLEMA DE INDEX
        cast_id.update({'movie_id': int(movie_id)})
        cast_id.pop('profile_path', None)
        movie_cast_df = movie_cast_df.append(cast_id, ignore_index=True)


{'gender': 2, 'credit_id': '52fe4214c3a36847f80024db', 'order': 0, 'movie_id': 77, 'cast_id': 4, 'character': 'Leonard', 'id': 529, 'name': 'Guy Pearce'}


/Users/raphael.prospero/.pyenv/versions/3.5.0/envs/pyenv-3.5.0/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


AttributeError: 'dict' object has no attribute 'dtype'

In [204]:
linha = movie_credits_df.iloc[0]

In [207]:
print(linha['cast'][0].dtype)

AttributeError: 'dict' object has no attribute 'dtype'

In [203]:
print(movie_credits_df['cast'][0][0])

{'gender': 2, 'credit_id': '52fe4214c3a36847f80024db', 'order': 0, 'movie_id': 77, 'cast_id': 4, 'character': 'Leonard', 'id': 529, 'name': 'Guy Pearce'}


In [190]:
movie_credits_df.shape[0]

147507